In [38]:
import os
import joblib
import random
import numpy as np
import pandas as pd
import datetime as dt
from tqdm import tqdm
import tensorflow as tf
from tensorflow import keras
from keras import layers, Model
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, MinMaxScaler

# Loading and Preparing Data

In [2]:
users = pd.read_csv('./Data/users.csv')
cards = pd.read_csv('./Data/cards.csv')
trans = pd.read_csv('./Data/transactions.csv')

In [3]:
users_needed_cols = ['Current Age', 'Retirement Age', 'Gender', 'Zipcode', 'Per Capita Income - Zipcode', 
                    'Yearly Income - Person', 'Total Debt', 'FICO Score', 'Num Credit Cards']
cards_needed_cols = ['User', 'CARD INDEX', 'Year PIN last Changed', 'Acct Open Date', 'Expires', 'Card Duration', 
                    'Has Chip', 'Cards Issued', 'Credit Limit', 'Visa', 'Mastercard', 'Discover', 'Amex', 'Debit', 
                    'Credit', 'Debit (Prepaid)']
trans_needed_cols = ['User', 'Card', 'Time', 'Time From Now', 'Amount', 'Merchant Name', 'Merchant City', 
                    'Merchant State', 'Zip', 'MCC', 'Errors?', 'Swipe Transaction', 'Online Transaction', 
                    'Chip Transaction', 'Is Fraud?']

In [4]:
data_prep_models = {}

## Handling Zip Codes

In [5]:
zip = pd.concat([users.Zipcode, trans.Zip.fillna(0).astype(int)])
zip_encoder = LabelEncoder()
zip_encoder.fit_transform(zip)
users.Zipcode = zip_encoder.transform(users.Zipcode)
trans.Zip = zip_encoder.transform(trans.Zip.fillna(0).astype(int))
data_prep_models['zip_encoder'] = zip_encoder

In [6]:
users['Zipcode'].min(), users['Zipcode'].max()

(33, 27247)

In [7]:
trans['Zip'].min(), trans['Zip'].max()

(0, 27321)

In [8]:
users['Zipcode'] = users['Zipcode'].astype(np.uint16)
trans['Zip'] = trans['Zip'].astype(np.uint16)

## Handling User Ages

In [9]:
users[['Current Age', 'Retirement Age']].min(), users[['Current Age', 'Retirement Age']].max()

(Current Age       18
 Retirement Age    50
 dtype: int64,
 Current Age       101
 Retirement Age     79
 dtype: int64)

In [10]:
users['Current Age'] = users['Current Age'].astype(np.uint8)
users['Retirement Age'] = users['Retirement Age'].astype(np.uint8)

## Handling User Gender

In [11]:
users['Gender'].value_counts()

Female    1016
Male       984
Name: Gender, dtype: int64

In [12]:
gender_encoder = LabelEncoder()
gender_encoder.fit_transform(users['Gender'])
users['Gender'] = gender_encoder.transform(users['Gender'])
data_prep_models['gender_encoder'] = gender_encoder

## Handling Money

In [13]:
users['Per Capita Income - Zipcode'] = users['Per Capita Income - Zipcode'].str.replace('$', '', regex=True).astype(int)
users['Yearly Income - Person'] = users['Yearly Income - Person'].str.replace('$', '', regex=True).astype(int)
users['Total Debt'] = users['Total Debt'].str.replace('$', '', regex=True).astype(int)

## Handling FICO Score

In [14]:
users['FICO Score'].min(), users['FICO Score'].max()

(480, 850)

In [15]:
users['FICO Score'] = users['FICO Score'].astype(np.uint16)

## Handling User's Credit Cards Number

In [16]:
users['Num Credit Cards'] = users['Num Credit Cards'].astype(np.uint8)

## Handling User and Card Index

In [17]:
cards['User'] = cards['User'].astype(np.uint16)
cards['CARD INDEX'] = cards['CARD INDEX'].astype(np.uint8)

## Handling Card Brand and Type

In [18]:
cards['Card Brand'].value_counts(), cards['Card Type'].value_counts()

(Mastercard    3209
 Visa          2326
 Amex           402
 Discover       209
 Name: Card Brand, dtype: int64,
 Debit              3511
 Credit             2057
 Debit (Prepaid)     578
 Name: Card Type, dtype: int64)

In [19]:
card_brand_type_encoder = OneHotEncoder(categories=[cards['Card Brand'].unique(), cards['Card Type'].unique()], handle_unknown='ignore', dtype=np.uint8)
brand = cards[['Card Brand', 'Card Type']]
card_brand_type_encoder.fit_transform(brand)
brand = pd.DataFrame(card_brand_type_encoder.transform(brand).toarray(), columns=['Visa', 'Mastercard', 'Discover', 'Amex', 'Debit', 'Credit', 'Debit (Prepaid)'])
cards = pd.concat([cards.drop(columns=['Card Brand', 'Card Type']), brand], axis=1)
data_prep_models['card_brand_type_encoder'] = card_brand_type_encoder
card_brand_type_encoder.categories_

[array(['Visa', 'Mastercard', 'Discover', 'Amex'], dtype=object),
 array(['Debit', 'Credit', 'Debit (Prepaid)'], dtype=object)]

## Handle Has Chip

In [20]:
cards['Has Chip'].value_counts()

YES    5500
NO      646
Name: Has Chip, dtype: int64

In [21]:
has_chip_encoder = LabelEncoder()
has_chip_encoder.fit_transform(cards['Has Chip'])
cards['Has Chip'] = has_chip_encoder.transform(cards['Has Chip'])
data_prep_models['has_chip_encoder'] = has_chip_encoder

## Handle Cards Issued

In [22]:
cards['Cards Issued'].min(), cards['Cards Issued'].max()

(1, 3)

In [23]:
cards['Cards Issued'] = cards['Cards Issued'].astype(np.uint8)

## Handle Card Credit Limit

In [24]:
cards['Credit Limit'] = cards['Credit Limit'].str.replace('$', '', regex=True).astype(int)

## Handle Cards Dates

In [25]:
now = dt.datetime.now()
cards['Year PIN last Changed'] += (now.year - cards['Year PIN last Changed']).astype(np.uint16)
cards['Expires'] = ((now - pd.to_datetime(cards['Expires'], infer_datetime_format=True)) / np.timedelta64(1, 'M')).astype(int)
cards['Acct Open Date'] = ((now - pd.to_datetime(cards['Acct Open Date'], infer_datetime_format=True)) / np.timedelta64(1, 'M')).astype(int)
cards['Card Duration'] = cards['Expires'] - cards['Acct Open Date']

## Transaction User and Card

In [26]:
trans.sort_values(by=['User', 'Card', 'Year', 'Month', 'Day', 'Time'], inplace=True)
trans['User'] = trans['User'].astype(np.uint16)
trans['Card'] = trans['Card'].astype(np.uint8)

## Handling Date and Time

In [28]:
trans['Time From Now'] = (now - pd.to_datetime(dict(year=trans['Year'], month=trans['Month'], day=trans['Day']))).dt.days
trans['Time'] = trans['Time'].str.replace(':[0-9]+', '', regex=True).astype(np.uint8)

## Handling Amount

In [29]:
trans['Amount'] = trans['Amount'].str.replace('$', '', regex=True).astype(float).apply(lambda x: np.log(max(1.0, x)))

## Handling Use Chip

In [30]:
use_chip_encoder = OneHotEncoder(categories=[list(trans['Use Chip'].unique())], handle_unknown='ignore', dtype=np.uint8)
use_chip_encoder.fit_transform(trans[['Use Chip']])
use_chip = pd.DataFrame(use_chip_encoder.transform(trans[['Use Chip']]).toarray(), columns=['Swipe Transaction', 'Online Transaction', 'Chip Transaction'])
trans = pd.concat([trans.drop(columns=['Use Chip']), use_chip], axis=1)
data_prep_models['use_chip_encoder'] = use_chip_encoder
use_chip_encoder.categories_

[array(['Swipe Transaction', 'Online Transaction', 'Chip Transaction'],
       dtype=object)]

## Handling Merchant City, State, Name and MCC

In [31]:
merchant_name_encoder = LabelEncoder()
merchant_name_encoder.fit(trans['Merchant Name'])
trans['Merchant Name'] = merchant_name_encoder.transform(trans['Merchant Name'])
data_prep_models['merchant_name_encoder'] = merchant_name_encoder

In [32]:
trans['Merchant Name'].min(), trans['Merchant Name'].max()

(0, 100342)

In [33]:
trans['Merchant Name'] = trans['Merchant Name'].astype(np.uint32)

In [34]:
merchant_city_encoder = LabelEncoder()
merchant_city_encoder.fit(trans['Merchant City'])
trans['Merchant City'] = merchant_city_encoder.transform(trans['Merchant City'])
data_prep_models['merchant_city_encoder'] = merchant_city_encoder

In [35]:
trans['Merchant City'].min(), trans['Merchant City'].max()

(0, 13428)

In [36]:
trans['Merchant City'] = trans['Merchant City'].astype(np.uint16)

In [37]:
trans['Merchant State'].fillna('Unknown', inplace=True)
merchant_state_encoder = LabelEncoder()
merchant_state_encoder.fit(trans['Merchant State'])
trans['Merchant State'] = merchant_state_encoder.transform(trans['Merchant State'])
data_prep_models['merchant_state_encoder'] = merchant_state_encoder

In [38]:
trans['Merchant State'].min(), trans['Merchant State'].max()

(0, 223)

In [39]:
trans['Merchant State'] = trans['Merchant State'].astype(np.uint8)

In [40]:
mcc_encoder = LabelEncoder()
mcc_encoder.fit(trans['MCC'])
trans['MCC'] = mcc_encoder.transform(trans['MCC'])
data_prep_models['mcc_encoder'] = mcc_encoder

In [41]:
trans['MCC'].min(), trans['MCC'].max()

(0, 108)

In [42]:
trans['Merchant State'] = trans['Merchant State'].astype(np.uint8)

## Handling Errors

In [43]:
trans['Errors?'].fillna('No Errors', inplace=True)
errors_encoder = LabelEncoder()
errors_encoder.fit(trans['Errors?'])
trans['Errors?'] = errors_encoder.transform(trans['Errors?'])
data_prep_models['errors_encoder'] = errors_encoder

In [44]:
trans['Errors?'].min(), trans['Errors?'].max()

(0, 23)

In [45]:
trans['Errors?'] = trans['Errors?'].astype(np.uint8)

In [46]:
trans['Is Fraud?'] = trans['Is Fraud?'].replace({'No': 0, 'Yes': 1}).astype(np.uint8)

## Saving Data Prep Models

In [47]:
joblib.dump(data_prep_models, './Models/DataPrepModels.pkl')

['./Models/DataPrepModels.pkl']

## Saving Final Data

In [55]:
dtypes = {}
dtypes['users'] = users[users_needed_cols].dtypes.to_dict()
dtypes['cards'] = cards[cards_needed_cols].dtypes.to_dict()
dtypes['trans'] = trans[trans_needed_cols].dtypes.to_dict()
joblib.dump(dtypes, './Data/FinalData/types.pkl')

['./Data/FinalData/types.pkl']

In [57]:
users[users_needed_cols].to_csv('./Data/FinalData/users.csv', index=False)
cards[cards_needed_cols].to_csv('./Data/FinalData/cards.csv', index=False)
trans[trans_needed_cols].to_csv('./Data/FinalData/trans.csv', index=False)

# Load Data

In [4]:
types = joblib.load('./Data/FinalData/types.pkl')

users_needed_cols = ['Current Age', 'Retirement Age', 'Gender', 'Zipcode', 'Per Capita Income - Zipcode', 
                    'Yearly Income - Person', 'Total Debt', 'FICO Score', 'Num Credit Cards']
cards_needed_cols = ['User', 'CARD INDEX', 'Year PIN last Changed', 'Acct Open Date', 'Expires', 'Card Duration', 
                    'Has Chip', 'Cards Issued', 'Credit Limit', 'Visa', 'Mastercard', 'Discover', 'Amex', 'Debit', 
                    'Credit', 'Debit (Prepaid)']
trans_needed_cols = ['User', 'Card', 'Time', 'Time From Now', 'Amount', 'Merchant Name', 'Merchant City', 
                    'Merchant State', 'Zip', 'MCC', 'Errors?', 'Swipe Transaction', 'Online Transaction', 
                    'Chip Transaction', 'Is Fraud?']

users = pd.read_csv('./Data/FinalData/users.csv', usecols=users_needed_cols, dtype=types['users'])
cards = pd.read_csv('./Data/FinalData/cards.csv', usecols=cards_needed_cols, dtype=types['cards'])
trans = pd.read_csv('./Data/FinalData/trans.csv', usecols=trans_needed_cols, dtype=types['trans'])

# Prepare Transactions Windows

In [ ]:
stride = 5
seq_len = 10
trans_df = []
targets = []
trans_feats = [
    'Time', 'Time From Now', 'Amount', 'Merchant Name', 'Merchant City', 'Merchant State', 
    'Zip', 'MCC', 'Errors?', 'Swipe Transaction', 'Online Transaction', 'Chip Transaction'
]
user_card_dict = trans.groupby('User')['Card'].unique().to_dict()

for user_id in tqdm(user_card_dict.keys()):
    for card_id in user_card_dict[user_id]:
        t_df = trans[(trans['User'] == user_id) & (trans['Card'] == card_id)]
        for i in range(0, t_df.shape[0], stride):
            wind_df = t_df[i:i+seq_len]
            trans_df.append((user_id, card_id, wind_df[trans_feats]))
            targets.append(wind_df['Is Fraud?'].iloc[-1])
joblib.dump(trans_df, '/content/drive/MyDrive/IBM_Dataset/trans_df.pkl')
joblib.dump(targets, '/content/drive/MyDrive/IBM_Dataset/targets.pkl')

In [ ]:
x_train_val, x_test, y_train_val, y_test = train_test_split(trans_df, targets, test_size=0.3, random_state=42, stratify=targets)
x_train, x_val, y_train, y_val = train_test_split(trans_df, targets, test_size=0.1, random_state=42, stratify=targets)

# Model Parameters

In [58]:
batch_size = 8
n_step = 10
users_feats = 9
cards_feats = 14
trans_feats = 12
n_feats = users_feats + cards_feats + trans_feats
n_feats

35

# Data Loader

In [ ]:
def get_batch(user_df: pd.DataFrame, card_df: pd.DataFrame, trans_df: list, target: list, batch_size: int, conv_input=False):
    card_df.set_index(['User', 'CARD INDEX'], inplace=True)
    while True:
        idx = np.random.choice(range(len(trans_df)), size=len(trans_df), replace=False)
        for i in range(0, len(idx), batch_size):
            t_df = [trans_df[idx[j]] for j in range(i, i + batch_size)]
            X = np.zeros((len(t_df), n_step, n_feats))
            Y = np.array([targets[idx[j]] for j in range(i, i + batch_size)])

            for it, (user_id, card_id, wind_df) in enumerate(t_df):
                user = user_df.iloc[user_id]
                card = card_df.loc[[user_id, card_id]]
                x = pd.concat([user, card, wind_df], axis=1).ffill().values
                X[it] = x
            if conv_input:
                X = np.transpose(X, axis=[0, 2, 1])
            yield X, Y

# Training Plots

In [ ]:
def plot_history(history):
    """Plots accuracy/loss for training/validation set as a function of the epochs
        :param history: Training history of model
        :return:
    """

    fig, axs = plt.subplots(4)
    fig.tight_layout(pad=3)

    axs[0].plot(history.history["loss"], label="train loss")
    axs[0].plot(history.history["val_loss"], label="val loss")
    axs[0].set_ylabel("loss")
    axs[0].set_xlabel("Epoch")
    axs[0].legend(loc="upper right")
    axs[0].set_title("Loss")

    axs[1].plot(history.history["auc"], label="train AUC")
    axs[1].plot(history.history["val_auc"], label="val AUC")
    axs[1].set_ylabel("AUC")
    axs[1].legend(loc="lower right")
    axs[1].set_title("AUC")

    axs[2].plot(history.history["precision"], label="train precision")
    axs[2].plot(history.history["val_precision"], label="val precision")
    axs[2].set_ylabel("Precision")
    axs[2].legend(loc="lower right")
    axs[2].set_title("Precision")

    axs[3].plot(history.history["recall"], label="train recall")
    axs[3].plot(history.history["val_recall"], label="val recall")
    axs[3].set_ylabel("Recall")
    axs[3].legend(loc="lower right")
    axs[3].set_title("Recall")


# Models

## LSTM

In [86]:
def LstmModel():
  model = keras.Sequential()
  
  model.add(layers.LSTM(units=64, kernel_regularizer='l2', return_sequences=True, time_major=False, input_shape=(n_step, n_feats)))
  model.add(layers.LSTM(units=32, kernel_regularizer='l2', return_sequences=True, time_major=False))
  model.add(layers.BatchNormalization())
  model.add(layers.LSTM(units=16, kernel_regularizer='l2', return_sequences=False, time_major=False))
  model.add(layers.Dense(16))
  model.add(layers.Dropout(0.2))
  model.add(layers.Dense(1, activation='sigmoid'))
  return model

# GRU

In [87]:
def GruModel():
  model = keras.Sequential()
  
  model.add(layers.GRU(units=64, kernel_regularizer='l2', return_sequences=True, time_major=False, input_shape=(n_step, n_feats)))
  model.add(layers.GRU(units=32, kernel_regularizer='l2', return_sequences=True, time_major=False))
  model.add(layers.BatchNormalization())
  model.add(layers.GRU(units=16, kernel_regularizer='l2', return_sequences=False, time_major=False))
  model.add(layers.Dense(16))
  model.add(layers.Dropout(0.2))
  model.add(layers.Dense(1, activation='sigmoid'))
  return model

# Conv1D

In [82]:
def ConvModel():
  model = keras.Sequential()
  
  model.add(layers.Conv1D(64, 3, activation='relu', kernel_regularizer='l2', input_shape=(n_feats, n_step)))
  model.add(layers.AveragePooling1D(pool_size=2, strides=2))
  model.add(layers.Conv1D(128, 7, activation='relu', kernel_regularizer='l2'))
  model.add(layers.Conv1D(512, 5, activation='relu', kernel_regularizer='l2'))
  model.add(layers.Conv1D(1024, 5, activation='relu', kernel_regularizer='l2'))
  model.add(layers.AveragePooling1D(pool_size=2))
  model.add(layers.Flatten())
  model.add(layers.Dense(256))
  model.add(layers.Dropout(0.3))
  model.add(layers.Dense(128))
  model.add(layers.Dense(1, activation='sigmoid'))
  return model

## Conv-LSTM

In [74]:
def ConvLstmModel():
  model = keras.Sequential()

  model.add(layers.Conv1D(64, 3, activation='relu', kernel_regularizer='l2', input_shape=(n_feats, n_step)))
  model.add(layers.AveragePooling1D(pool_size=2, strides=2))
  model.add(layers.Conv1D(128, 3, activation='relu', kernel_regularizer='l2'))
  model.add(layers.Conv1D(512, 5, activation='relu', kernel_regularizer='l2'))
  model.add(layers.Conv1D(1024, 5, activation='relu', kernel_regularizer='l2'))

  model.add(layers.Dense(256))
  model.add(layers.Dropout(0.3))

  model.add(layers.LSTM(64, kernel_regularizer='l2',  return_sequences=True))
  model.add(layers.LSTM(64, kernel_regularizer='l2',  return_sequences=False))
  
  model.add(layers.Dense(256))
  model.add(layers.Dense(128))
  model.add(layers.Dense(1, activation='sigmoid'))
  return model

# Auto-Encoder

## Conv1D

In [101]:
def ConvAE():
    input = layers.Input(shape=(n_feats, n_step))
    x = layers.Conv1D(64, 3, strides=2, activation='relu', kernel_regularizer='l2')(input)
    x = layers.Conv1D(128, 5, strides=2, activation='relu', kernel_regularizer='l2')(x)
    x = layers.AveragePooling1D(4)(x)
    y = layers.Conv1DTranspose(128, 1, strides=7, activation='relu', kernel_regularizer='l2')(x)
    y = layers.Conv1DTranspose(64, 5, strides=2, activation='relu', kernel_regularizer='l2')(y)
    y = layers.Conv1DTranspose(10, 3, strides=2, activation='relu', kernel_regularizer='l2')(y)
    encoder = Model(inputs=input, outputs=x)
    ae = Model(inputs=input, outputs=y)
    return ae

## LSTM

In [104]:
def LstmAE():
    input = layers.Input(shape=(n_step, n_feats))
    x = layers.LSTM(units=64, kernel_regularizer='l2', return_sequences=True)(input)
    x = layers.LSTM(units=32, kernel_regularizer='l2', return_sequences=False)(x)
    y = layers.RepeatVector(n_step)(x)
    y = layers.LSTM(units=32, kernel_regularizer='l2', return_sequences=True)(y)
    y = layers.LSTM(units=64, kernel_regularizer='l2', return_sequences=True)(y)
    output = layers.TimeDistributed(layers.Dense(n_feats))(y)
    encoder = Model(inputs=input, outputs=x)
    ae = Model(inputs=input, outputs=output)
    return ae, encoder

# Model Definition

In [105]:
def get_model(model_name):
    model = None
    if model_name == 'LSTM':
        model = LstmModel()
    elif model_name == 'GRU':
        model = GruModel()
    elif model_name == 'CONV':
        model = ConvModel()
    elif model_name == 'CONVLSTM':
        model = ConvLstmModel()
    elif model_name == 'CONVAE':
        model = ConvAE()
    elif model_name == 'LSTMAE':
        model = LstmAE()
    return model

# Training

In [ ]:
lr = 0.001
epochs = 20
weight_decay = 0.0
checkpoint_dir = '/content/drive/MyDrive/IBM_Dataset/checkpoints'

In [ ]:
model_name = ''
model = get_model(model_name)
class_weights = compute_class_weight(class_weight='balanced', classes=[0, 1], y=targets)
class_weights = {0: class_weights[0], 1: class_weights[1]}

In [ ]:
train_loader = get_batch(users, cards, x_train, y_train, batch_size, True if model_name in {'CONV', 'CONVLSTM', 'CONVAE'} else False)
val_loader = get_batch(users, cards, x_val, y_val, batch_size, True if model_name in {'CONV', 'CONVLSTM', 'CONVAE'} else False)
test_loader = get_batch(users, cards, x_test, y_test, batch_size, True if model_name in {'CONV', 'CONVLSTM', 'CONVAE'} else False)

In [ ]:
model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=lr, weight_decay=weight_decay), 
    loss='binary_crossentropy',
    loss_weights=class_weights,
    metrics=[keras.metrics.AUC()],
    weighted_metrics=[keras.metrics.Precision(), keras.metrics.Recall()]
)
train_details = model.fit(
    x=train_loader,
    epochs=epochs,
    validation_data=val_loader,
    callbacks=[
        tf.keras.callbacks.ReduceLROnPlateau(factor=0.5, patience=3, monitor='val_loss'),
        tf.keras.callbacks.ModelCheckpoint(filepath=os.path.join(checkpoint_dir, model_name), monitor='val_loss', save_best_only=True)
    ],
    class_weight=class_weights
)